# はじめに

## このノートブックについて

このサンプルコードはTEIファイルを使って何らかのテキスト分析を行う際のヒントです。
まずは、どんな風にコードが動作するのか、各コードセルの実行ボタン（▶）を押して確認してください。


## このプロジェクトについて

大阪大学グローバル日本学教育研究拠点

2023年度 教育プログラム開発型

[拠点形成プロジェクト「人文科学分野向け研究データ管理を促進するデジタル・ヒューマニティーズ学習教材開発」](https://www.gjs.osaka-u.ac.jp/project/2023a/)



## ライセンスについて
ライセンスはクリエイティブコモンズライセンス[表示 4.0 国際 (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/deed.ja)です。ぜひ内容をアレンジして自分の目的を達成してください。



# 環境設定とデータの読み込み


まず、動作に必要なプログラムをインストールしましょう。
そして、コード左端にある実行ボタン(▶)をクリックしてください。

BeautifulSoupは、PythonでHTMLやXMLファイルを簡単に解析（パース）するためのライブラリ（まとまったプログラムのセット）です。Webスクレイピングやデータの抽出、文書の構造解析に便利です。

In [ ]:
!pip install beautifulsoup4


次に、TEIデータを読み込みます。
TEI形式はXML形式でもあります。そのためファイルの拡張子はxmlです。このサンプルコードでは[青空文庫に格納されている、太宰治作「走れメロス」](https://www.aozora.gr.jp/cards/000035/files/1567_14913.html)のTEIデータを使います。




In [ ]:
!curl -O 'https://raw.githubusercontent.com/nikolito/TEI-tools/main/meros.xml'

ダウンロードしたデータは、この画面の左端にフォルダの形をしたアイコンの中にあります。

# TEI/XMLファイルの中身を見てみよう

ファイルから必要な情報を抽出するためには、まずは分析対象のTEI/XMLファイルの構造を調べる必要があります。

早速、さきほどダウンロードしたTEI/XMLファイルを読みだしてみましょう。

In [ ]:
with open("meros.xml", encoding='utf8') as f:
    print(f.read())

# TEIファイルからタグを除去したテキストを表示しよう



In [ ]:
#モジュールのインストール
from bs4 import BeautifulSoup

# TEIファイルを読み込んで、beautifulsoupがxml要素を取り出す
file_path = "meros.xml"

with open(file_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "xml")

# <rt>タグを削除する
for ruby in soup.find_all("rt"):
    ruby.decompose()

# <text>要素を取り出して、その中のタグを除去（タグなしの本文テキストにする）
text_content = soup.find("text").get_text(strip=True)

# 最初の1000文字をタグなしテキストから表示する
print(text_content[:1000] + "...")

改行がないので、少しレイアウトが崩れています。

textwrapライブラリを使ってもう少し読みやすく形を整えてみます。

In [ ]:
# 20文字で折り返し、300字まで表示してみる
import textwrap

wrapped_text = textwrap.fill(text_content, width=20)
print(wrapped_text[:300] + "...")


ここから先では、XMLファイルのなかでご自身が必要とする情報のみを取り出す方法を見ていきます。そのためには、マークアップルールを確認することが極めて重要な作業ステップになります。

# このファイルのマークアップルールを確認するには

## 階層構造を調べよう

TEIファイルの中から必要な情報を出力する際には、どんなタグが使われているかと、その階層構造を調べる必要があります。

まずはタグの階層構造を簡単に把握できる便利なプログラムを紹介します。

In [ ]:
from bs4 import BeautifulSoup
from IPython.display import display, HTML

def generate_collapsible_html(tag, indent=0):
    # 再起的にxmlタグを階層表示します
    children = tag.find_all(recursive=False)
    if not children:
        return f'<div style="margin-left: {indent}px;">{tag.name}</div>'

    inner_html = "".join([generate_collapsible_html(child, indent + 20) for child in children])
    return f"""
    <details style="margin-left: {indent}px;">
        <summary>{tag.name}</summary>
        {inner_html}
    </details>
    """

# XML要素をsoupに入れます
file_path = "meros.xml"
with open(file_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "xml")

# HTML表示でタグの階層を開閉します
root = soup.find(True)  # 最初のタグを取得
html_output = generate_collapsible_html(root)
display(HTML(html_output))


↑上記セルを実行すると出力表示にTEIタグの階層構造が表示されます。 TEI > teiHeader > encodingDesc と順番に▶︎マークをクリックしていきましょう。
encodingDesc > editorialDecl > p > gi という階層になっていますね。

**XML形式のデータは必ず入れ子になっている**ので、プログラムで欲しいデータがある階層に辿り着くのが比較的簡単です。

## editorialDeclタグの調査

meros.xmlのeditorialDeclタグには、マークアップルールが記されているので抽出してみましょう。


In [ ]:
#モジュールのインストール
from bs4 import BeautifulSoup
import textwrap

# TEIファイルを読み込んで、beautifulsoupがxml要素を取り出す
file_path = "meros.xml"
with open(file_path, "r", encoding="utf-8") as file:
  soup = BeautifulSoup(file, "xml")

# マークアップルールを記した<editorialDecl>を選択して、変数editoricalDeclに代入
editorialDecl = soup.select('editorialDecl')

# editorialDeclの中身を出力する
wrapped_text = textwrap.fill(editorialDecl[0].get_text(strip=True), width=50) # widthでお好みの幅に変えてください
print(wrapped_text.replace("  ", ""))


## 上の出力結果の解説

\<editorialDecl>のなかには、以下のタグが入っています。このファイルでは、マークアップを行った人がplaceName、personName、rsタグを使って、名詞を場所、人名、人称代名詞の種類に分別しています。加えて、type属性でplaceNameにtype=realあるいはunrealを追加できるように定義しています。つまり、TEIマークアップを行った人が、**このファイルを使う全ての人に向けて**、editorialDeclタグの中で、それぞれのタグの持つ意味で名詞をテキストから抽出できるように定義を説明しています。

- \<p>（[散文における段落を示すタグ](https://www.tei-c.org/release/doc/tei-p5-doc/en/html/ref-p.html)）
- \<placeName>（[場所名](https://www.tei-c.org/release/doc/tei-p5-doc/en/html/ref-list.html)）
- \<personName>（[人名](https://www.tei-c.org/release/doc/tei-p5-doc/en/html/ref-item.html)）
- \<rs>（人称代名詞）

また、すべてのタグには**corresp**属性を付けて、名寄せを可能にしています。例えば、人称代名詞で「彼」と書かれているのがメロスであるならば、\<rs corresp="#メロス">彼</rs>とマークアップします。

- \#は同一文書内の要素を、テキスト内で参照できる**一意のID**で指定することを意味します。

- 参照先は、本文を表す\<text>タグ内ではなく、後付や参考文献、補足などを表記する**\<back>**タグの中に記載します。\<back>タグは、\<text>タグより後の同じ階層に置き、その中にIDのリストを以下の例の要領で記述していきます。

- 参照先の要素には**xml:id**属性を付与する必要があります。また、xml:idは文書内で一意でなければなりません。

```
（<text>内）
<persName corresp="#メロス">メロス</persName>
<placeName corresp="#シラクス">シラクス</placeName>

（<back>内）
<listPerson>
  <person xml:id="メロス">
    <persName>メロス</persName>
  </person>
</listPerson>
<listPlace>
  <place xml:id="シラクス">
    <placeName>シラクス</placeName>
  </place>
</listPlace>
```

# 名寄せをしよう

このTEIファイルでは、人名と地名の名寄せがすでに行われています。早速主人公のメロスが作中でどのような呼ばれ方をしているのかをリストにしてみましょう。そして、各呼び名の回数も表示してみましょう。

In [ ]:
from bs4 import BeautifulSoup
from collections import Counter

# TEIファイルを読み込んで、BeautifulSoupで解析
file_path = "meros.xml"  # 実際のパスを指定
with open(file_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "xml")

# <persName>タグのうちcorresp="#メロス"に該当する要素を抽出
melos_names = [pers.text for pers in soup.find_all("persName", {"corresp": "#メロス"})]

# 呼び名とその出現回数をカウント
name_counts = Counter(melos_names)

# 結果を表示
for name, count in name_counts.items():
  print(f"{name}: {count}回")


ちょっと、表示とカウントがおかしいですね。生のテキストにタグを付けていくので、表示がおかしくなることはよくあります。

余計なスペースと改行を名前から除去してカウントし直しましょう。

In [ ]:
from bs4 import BeautifulSoup
from collections import Counter

# TEIファイルを読み込んで、BeautifulSoupで解析
file_path = "meros.xml"
with open(file_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "xml")

# <persName>タグのうちcorresp="#メロス"に該当する要素を抽出
melos_names = []

for pers in soup.find_all("persName", {"corresp": "#メロス"}):
    # <rb>タグがある場合はその内容を抽出、ない場合は全体のテキストを使用
    if pers.find("rb"):
        melos_names.append(''.join(rb.text for rb in pers.find_all("rb")).strip())
    else:
        melos_names.append(pers.text.strip())

# 呼び名とその出現回数をカウント
name_counts = Counter(melos_names)

# 結果を表示
for name, count in name_counts.items():
    print(f"{name}: {count}回")


上記の改良したコードで各呼び名の使用回数を正しくカウントしました。17種類の呼び名がありますが、すべてメロスのことを指しています。

名寄せすると、登場人物が周囲からどんな風に思われていたのかや、作中での変化を読み取れそうです。

## パラグラフごとにメロスの呼称を観察しよう

今度はパラグラフごとにメロスの呼ばれ方を見てみましょう。

In [ ]:
from bs4 import BeautifulSoup
from collections import defaultdict
from IPython.display import display, Markdown

# TEIファイルを読み込んで、BeautifulSoupで解析
file_path = "meros.xml"  # 実際のパスを指定
with open(file_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "xml")

# text > body > pタグについて、各<p>タグごとにメロスの呼び名を抽出する
paragraphs = soup.find("text").find("body").find_all("p")
melos_names_by_paragraph = defaultdict(list)

for i, p in enumerate(paragraphs, start=1):
    for pers in p.find_all("persName", {"corresp": "#メロス"}):
        if pers.find("rb"):
            name = ''.join(rb.text for rb in pers.find_all("rb")).strip()
        else:
            name = pers.text.strip()
        melos_names_by_paragraph[i].append(name)

# 結果を表示
for para_num, names in melos_names_by_paragraph.items():
    display(Markdown(f"### Paragraph {para_num}"))
    display(Markdown(", ".join(names) if names else "--"))


この物語の主要人物はメロス、ディオニス（王）、セリヌンティウス（メロスの親友）です。

今度は、メロス、ディオニス、セリヌンティウスの呼称をパラグラフごとに抽出して一覧表を作ってみましょう。

## Pandasのデータフレームとは？

一覧表のような2次元の表を作るには、pandasというライブラリのデータフレームを使って表示するのが一般的です。

Pandasの**データフレーム**は、Pythonでデータを扱うための非常に便利な構造です。スプレッドシートのように、行と列で構成された2次元のデータを簡単に操作できます。

---

### 1. **データフレームの基本構造**
- **行（Row）**：個別のデータエントリ（レコード）
- **列（Column）**：各データの属性（特徴や値）
- **インデックス（Index）**：各行に割り当てられたラベル（デフォルトでは0から始まる連番）

| Index | Name    | Age | City      |
|-------|---------|-----|-----------|
| 0     | Alice   | 24  | New York  |
| 1     | Bob     | 27  | Los Angeles |
| 2     | Charlie | 22  | Chicago    |


In [ ]:
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
from IPython.display import display

# TEIファイルを読み込んで、BeautifulSoupで解析
file_path = "meros.xml"  # 実際のパスを指定
with open(file_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "xml")

for ruby in soup.find_all("rt"):
    ruby.decompose()

# ターゲットとするキャラクター
characters = {
    "#メロス": "メロス",
    "#ディオニス": "ディオニス",
    "#セリヌンティウス": "セリヌンティウス"
}

# 各<p>タグごとにキャラクターの呼び名を抽出する
paragraphs = soup.find("text").find("body").find_all("p")
data = defaultdict(lambda: defaultdict(list))
paragraph_texts = {}

for i, p in enumerate(paragraphs, start=1):
    paragraph_texts[i] = p.get_text(strip=True).replace("\n", "").replace("\r", "").replace(" ", "")[:100]  # 変な改行とスペースが入らないようにした上で、各パラグラフの最初の100文字を取得
    for pers in p.find_all("persName"):
        corresp = pers.get("corresp")
        if corresp in characters:
            name = ''.join(rb.text for rb in pers.find_all("rb")).strip() if pers.find("rb") else pers.text.strip()
            data[i][characters[corresp]].append(name)

# DataFrameに変換し、各呼び名を「、」区切りで表示
df = pd.DataFrame({char: ["、".join(data[para][char]) for para in range(1, len(paragraphs) + 1)] for char in characters.values()})
df.index = [f"P{i}" for i in range(1, len(paragraphs) + 1)]

# パラグラフのテキスト列を追加
df['Text (first 100 chars)'] = [paragraph_texts[para] for para in range(1, len(paragraphs) + 1)]

# 表を表示
display(df)


どの場面でどのような呼ばれ方をしているのか、この表で確認できるようになりました。

この表をスプレッドシートに貼り付けられるようにTSVデータとしてダウンロードできるようにしましょう。

TSVとはTab-Separated Valuesの略称です。Tabで表の項目を区切ります。また、一つのレコードは1行で表し、改行で区切ります。

もしTabでなくコンマで区切ったデータが欲しい時は以下のコードのsep='\t'のところをsep=','に書き換えて実行してください。

In [ ]:
def get_tsv_data(df):
  """
  PandasのdataframeをTSV形式の文字列に変換する関数です。
  """
  return df.to_csv(sep='\t', index=True)
  # indexが不要な場合は、index=Falseとします。

# 上記get_tsv_data(df)を呼び出してください。
tsv_data = get_tsv_data(df)

# ファイルを保存します。output.tsvの名前でmeros.xmlがある場所に保存されています。:
with open('output.tsv', 'w', encoding='utf-8') as f:
  f.write(tsv_data)

# 登場人物の共起ネットワーク分析

作品中で特定の人物がどの程度一緒に登場するかをパラグラフごとに分析してみましょう。

手順は以下の通りです。

1. 各パラグラフで登場するキャラクター（\<persName>）を抽出。
2. パラグラフ単位で共に登場するキャラクターidのペアを集計。
3. ネットワークグラフを作成し、共起頻度をエッジの太さで表現。

## 準備

最終的に人物ネットワーク図を生成します。
そのために必要なライブラリと日本語フォントを読み込みます。

（主要ライブラリ）
1. **networkx**: Pythonでグラフ（ネットワーク）構造を扱うための強力なライブラリです。ノード（点）とエッジ（線）で構成されるネットワークを作成、操作、分析、そして可視化するために設計されています。
2. **matplotlib**: Pythonでデータの可視化を行うための代表的なライブラリです。特に、2Dグラフの作成に優れており、シンプルな線グラフから複雑なヒートマップや3Dプロットまで、多様なグラフを描画できます。

## 物語中の登場人物を抽出

idと実際の呼ばれ方は\<text> > \<back> > \<listPerson>に入っています。

変数のall_characterに整理し、リストにしました。

In [ ]:
# TEIファイルを読み込んでBeautifulSoupで解析
file_path = "meros.xml"  # 実際のパスを指定
with open(file_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "xml")

# listPersonから登場人物IDと名前を抽出
all_characters = {}
for person in soup.find_all("person"):
    person_id = person.get("xml:id")
    pers_name = person.find("persName")
    if person_id and pers_name:
        name = ''.join(rb.text for rb in pers_name.find_all("rb")).strip() if pers_name.find("rb") else pers_name.text.strip()
        all_characters[person_id] = name

for character in all_characters:
  print(f"{character}: {all_characters[character]}")

次に、上記リストの人物の組み合わせがどのパラグラフに入っているかを調べます。

In [ ]:
# パラグラフごとの共起データを収集
paragraphs = soup.find("text").find("body").find_all("p")
cooccurrence = defaultdict(list)

for para_num, p in enumerate(paragraphs, start=1):
    present_characters = set()

    for pers in p.find_all("persName"):
        corresp = pers.get("corresp")
        if corresp is not None:
            corresp = corresp.strip("#")
            if corresp in all_characters:
                present_characters.add(corresp)

    # 共起をパラグラフ別に記録
    for char1 in present_characters:
        for char2 in present_characters:
            if char1 != char2:
                cooccurrence[(char1, char2)].append(para_num)

print(f"共起ペアの数: {len(cooccurrence)}")
print("共起ペア : 共起したパラグラフ番号")
for occurrence in cooccurrence:
  print(f"{occurrence}: {cooccurrence[occurrence]}")

共起したペアとそのパラグラフ番号がわかりました。

これをNetworkXというライブラリを使って画面に描画してみましょう。まずはNetworkX自体を説明します。

## NetworkXとは？

**NetworkX**は、Pythonでグラフ（ネットワーク）構造を扱うためのライブラリです。ノード（頂点）とエッジ（辺）を使って、さまざまな種類のグラフを作成・操作・解析できます。社会ネットワーク分析、交通ネットワーク、知識グラフなど、幅広い用途に利用されます。

以下のコードを実行させて、その次の具体的な解説を読んでください。

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# グラフの作成
G = nx.Graph()

# ノードの追加
G.add_node("A") #
G.add_node("B")

# エッジの追加
G.add_edge("A", "B", weight=3)

# ノードとエッジの表示
print("Nodes:", G.nodes())
print("Edges:", G.edges(data=True))

# グラフの可視化
nx.draw(G, with_labels=True)
plt.show()

## コードの説明

上記のコードをもとに、NetworkXの基本的な使い方を説明します。

---

### 1. **ライブラリのインポート**
```python
import networkx as nx
import matplotlib.pyplot as plt
```
- **`networkx`**: グラフを作成・操作するためのライブラリ。
- **`matplotlib.pyplot`**: グラフを可視化するために利用します。

---

### 2. **グラフの作成**
```python
G = nx.Graph()
```
- **`nx.Graph()`**: 無向グラフを作成します。ノード同士の関係が双方向の場合に使用します。
  - 他にも、有向グラフ（`DiGraph`）、重み付きグラフなどがあります。

---

### 3. **ノードの追加**
```python
G.add_node("A")
G.add_node("B")
```
- **`add_node`**: ノード（頂点）を追加します。
  - `"A"` や `"B"` はノードのラベル。
  - 数値や文字列など、さまざまなデータ型をノードとして使えます。

---

### 4. **エッジの追加**
```python
G.add_edge("A", "B", weight=3)
```
- **`add_edge`**: ノード間にエッジ（辺）を追加します。
  - ここではノード `"A"` と `"B"` をつなぐエッジを追加。
  - **`weight`** パラメータでエッジに重み（数値）を設定しています。

---

### 5. **ノードとエッジの表示**
```python
print("Nodes:", G.nodes())
print("Edges:", G.edges(data=True))
```
- **`G.nodes()`**: グラフ内のすべてのノードを取得します。
- **`G.edges(data=True)`**: グラフ内のすべてのエッジを取得し、エッジの属性（ここでは `weight`）も表示します。

**出力例**:
```
Nodes: ['A', 'B']
Edges: [('A', 'B', {'weight': 3})]
```

---

### 6. **グラフの可視化**
```python
nx.draw(G, with_labels=True)
plt.show()
```
- **`nx.draw`**: グラフを描画します。
  - **`with_labels=True`**: ノードにラベルを表示。
- **`plt.show()`**: グラフを描画ウィンドウに表示します。

---

### 実行結果
1. ノード "A" と "B" が描画され、両者をつなぐエッジが表示されます。
2. エッジには重み `weight=3` が設定されていますが、デフォルトでは描画には反映されません。カスタマイズすれば、エッジのラベルとして重みを表示することも可能です。

---

### NetworkXの特徴
- **簡単にグラフを作成**: ノードとエッジを柔軟に追加できます。
- **豊富な分析機能**: 最短経路、クラスタリング係数、ページランクなど。
- **多様なグラフ種類**: 無向グラフ、有向グラフ、重み付きグラフ、マルチグラフなど。
- **可視化**: MatplotlibやGraphvizと連携してネットワーク構造を視覚的に確認できます。

---

NetworkXは、グラフ理論を用いたデータのモデリングと分析に非常に適したライブラリです。

基本操作から始めて少しずつ応用的な解析に進めると良いでしょう。

では、いよいよnetworkxを使って「走れメロス」の共起ペアとその共起頻度をパラグラフ別に色分けして表示します。まずは、日本語で表示するには日本語のフォントをダウンロードする必要があります。

## NetworkXを使ったパラグラフ別登場人物の共起の可視化

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from collections import defaultdict
import matplotlib.font_manager as fm
import matplotlib.patches as mpatches

# 日本語フォントの設定（Google Colabの場合には適切なフォントを利用）
# ColabではIPAexGothicなどが利用可能
! curl -O "https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/Japanese/NotoSansCJKjp-Regular.otf"

font_path = 'NotoSansCJKjp-Regular.otf'  # Update with your font file path
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = font_prop.get_name()

上記が実行できたら、最後の実行ボタンを押して共起ネットワークを可視化してください。

In [ ]:
# ネットワークグラフの作成
G = nx.Graph()

for (char1, char2), paras in cooccurrence.items():
    G.add_edge(char1, char2, paragraphs=paras, weight=len(paras))  # エッジの太さは共起パラグラフ数に比例

# ノードラベルを対応する名前に変更
labels = {node: node for node in G.nodes}

# 色をパラグラフごとに割り当てる
colors = plt.cm.tab20.colors  # 20色のカラーマップ
edge_colors = []
color_map = {}

for _, _, data in G.edges(data=True):
    # パラグラフ番号で色を決定
    para_index = data['paragraphs'][0] % len(colors)
    edge_colors.append(colors[para_index])
    color_map[data['paragraphs'][0]] = colors[para_index]  # パラグラフ番号に対応する色を記録

# ノードの位置を決める
pos = nx.spring_layout(G, seed=42)

# ノードとエッジの描画
plt.figure(figsize=(14, 12))
nx.draw_networkx_nodes(G, pos, node_size=3000, node_color="lightblue")
nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=[G[u][v]['weight'] for u, v in G.edges()], alpha=0.7)
nx.draw_networkx_labels(G, pos, labels=labels, font_size=12, font_color="black", font_weight="bold", font_family=font_prop.get_name())

# 凡例を作成
legend_handles = [mpatches.Patch(color=color, label=f"Paragraph {para_num}") for para_num, color in color_map.items()]
plt.legend(handles=legend_handles, loc="upper left", title="Paragraph Colors", fontsize=10, title_fontsize=12, prop=font_prop)

# タイトル
plt.title("登場人物の共起ネットワーク (パラグラフ別色分け + エッジ太さは共起数)", fontsize=20, fontproperties=font_prop)
plt.axis('off')
plt.show()

## 可視化した共起ネットワーク図の読み方

### 図の構成と解釈

1. **ノード（青い円）**:
   - 各ノードは登場人物を表します。ノードラベルにはそれぞれの人物の名前が日本語で表示されています。
   - 例えば、「メロス」「ディオニス」「セリヌンティウス」といった登場人物が含まれています。

2. **エッジ（線）**:
   - エッジは、登場人物同士が同じパラグラフで共に登場したこと（共起）を表します。
   - **エッジの太さ**: 共起の頻度を示し、太いほど多くのパラグラフで一緒に登場していることを意味します。

3. **エッジの色**:
   - エッジの色は、どのパラグラフで共起が発生したかを示しています。
   - 各色は特定のパラグラフ番号に対応しており、凡例（legend）にその対応が表示されています。

4. **凡例（Legend）**:
   - パラグラフ番号ごとにエッジの色が割り当てられています。
   - 例えば、「Paragraph 1」が赤、「Paragraph 2」が青などのように、各パラグラフでの共起を色で簡単に識別できます。

---

### どのように読むか？
- **太くてカラフルなエッジ**:
  - 複数のパラグラフで共起が発生している登場人物のペア。重要な関係性を示している可能性があります。
  - 例: メロスとセリヌンティウスのエッジが太く、色が多ければ、物語の中で彼らが何度も共起していることを意味します。

- **細いエッジ**:
  - 一部のパラグラフでのみ共起が発生している登場人物のペア。関係が限定的であることを示します。

- **色の多さ**:
  - エッジに多くの色が使われている場合、登場人物同士が物語全体を通して頻繁に共起していることを示しています。

---

### この図で得られる洞察
- **主要な登場人物の関係**:
  - 太くカラフルなエッジは、物語の中で特に重要な関係性を示します。
  - 例: メロスとディオニス、メロスとセリヌンティウスなど。

- **パラグラフごとの特徴**:
  - 各パラグラフでどの登場人物が一緒に登場しているかを視覚的に把握できます。
  - 物語の展開に伴う人物関係の変化を追跡するのに役立ちます。

---

この図は物語の人物関係を視覚化する強力なツールです。より詳細な分析が必要な場合、各パラグラフの具体的な内容と照らし合わせて関係性を深掘りすることができます。